# Clean up
This notebook provides code for end-to-end clean up of your SageMaker environment and preparation for stack deletion.

<div class="alert alert-info"> ❗ This is a <b>destructive action</b>. All SageMaker data, notebooks, projects, artifacts and correspoinding S3 buckets will be deleted! All data on the EFS will be deleted (SageMaker home directories). You may want to backup the EFS before deletion❗ 
</div>

## Load environment variables
If you haven't done so, you have to run [00-setup notebook](00-setup.ipynb) first!

In [ ]:
import boto3
import json
import sagemaker

sm = boto3.client("sagemaker")
cf = boto3.client("cloudformation")

In [ ]:
%store -r domain_id
%store -r security_group_ids
%store -r subnets
%store -r processing_role
%store -r execution_role
%store -r model_execution_role
%store -r pipeline_role
%store -r data_bucket
%store -r model_bucket
%store -r ebs_key_id
%store -r s3_key_id
%store -r network_config
%store -r env_name
%store -r env_type
%store -r env_type_staging_name
%store -r env_type_prod_name

try:
    domain_id
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-SETUP notebook     ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

## Step1: Clean up MLOps projects

### Enumerate projects and select projects to be deleted

In [ ]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

In [ ]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"The following projects will be deleted: {json.dumps(projects_to_delete, indent=2)}")

### Remove CloudFormation stack sets
This code removes stack sets which were used for model deployment

In [ ]:
import time

for p in projects_to_delete:
    for ss in [
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-{env_type_staging_name}",
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-{env_type_prod_name}"
            ]:
        try:
            accounts = [a["Account"] for a in cf.list_stack_instances(StackSetName=ss)["Summaries"]]
            print(f"delete stack set instances for {ss} stack set for the accounts {accounts}")
            r = cf.delete_stack_instances(
                StackSetName=ss,
                Accounts=accounts,
                Regions=[boto3.session.Session().region_name],
                RetainStacks=False,
            )
            print(r)
            time.sleep(180)
        except Exception:
            pass

        print(f"delete stack set {ss}")
        r = cf.delete_stack_set(
            StackSetName=ss
        )
        print(r)

### Delete projects

In [ ]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

### Empty project S3 buckets

In [ ]:
for p in projects_to_delete:
    !aws s3 rb s3://sm-mlops-cp-{p['ProjectName']}-{p['ProjectId']} --force    

## Step 2: Empty data and model S3 buckets

<div class="alert alert-info"> 
❗ This is a <b>destructive action</b>. All SageMaker data, logs, and model artifacts in data and models S3 buckets will be deleted❗
</div>

In [ ]:
!aws s3 rm s3://{data_bucket} --recursive
!aws s3 rm s3://{model_bucket} --recursive 

## Step 3: Generate clean up CLI commands 
This section generates the clean-up CLI commands based on your environment. You must run this commands in your CLI terminal in the solution working directory (git clone directory) after you close Studio.

In [ ]:
def print_aws_cli(stackname):
    if type(stackname) == list:
        stackname = stackname[0]['StackName'] if len(stackname) else 'None'

    if stackname and stackname != "None":
        ## condition for Service Catalog deployment-type
        if f"SC-{sagemaker.get_execution_role().split(':')[4]}" in stackname: 
            print(f"pipenv run python3 functions/pipeline/terminate-sc-product-cli.py {('-').join(stackname.split('-')[-2:])}")
        else:
            print(f"aws cloudformation delete-stack --stack-name {stackname}")
            print(f"aws cloudformation wait stack-delete-complete --stack-name {stackname}")

In [ ]:
# find data science environment stack based on the domain id
env_stackname = [
    s for s in cf.list_stacks(StackStatusFilter=["CREATE_COMPLETE"])["StackSummaries"] 
        if not s.get("ParentId") and cf.describe_stacks(StackName=s["StackName"])["Stacks"][0].get("Outputs") 
            and len([o for o in cf.describe_stacks(StackName=s["StackName"])["Stacks"][0]["Outputs"] 
                if o["OutputKey"] == "SageMakerDomainId" and o["OutputValue"] == domain_id]) != 0
]

# find core infrastructure stack name based on the output name 
core_stackname = [
    s for s in cf.list_stacks(StackStatusFilter=["CREATE_COMPLETE"])["StackSummaries"] 
        if not s.get("ParentId") and cf.describe_stacks(StackName=s["StackName"])["Stacks"][0].get("Outputs") 
            and len([o for o in cf.describe_stacks(StackName=s["StackName"])["Stacks"][0]["Outputs"] 
                if o["OutputKey"] == "SCLaunchRoleArn"]) != 0
]

# find (optional) CloudFormation package stack name
package_cfn_stackname = [
    s for s in cf.list_stacks(StackStatusFilter=["CREATE_COMPLETE"])["StackSummaries"] 
        if cf.describe_stacks(StackName=s["StackName"])["Stacks"][0].get("Outputs") 
            and len([o for o in cf.describe_stacks(StackName=s["StackName"])["Stacks"][0]["Outputs"] 
                if o["OutputKey"] == "StartBuildCLICommand"]) != 0
]

print(f"Data science environment stack name: {env_stackname}")
print(f"Core infrastructure stack name: {core_stackname}")
print(f"CloudFormation template package stack name: {package_cfn_stackname}")

Copy the output of the following code cell and run it in your CLI terminal in the solution directory **after** you close Studio.

<div class="alert alert-info"> ❗ The follwing code contains AWS CLI command to <b>delete SageMaker EFS</b>. All data in user home directories will be deleted when you run this code in your CLI terminal❗ 
</div>

In [ ]:
print("*********************************************************")
print("* COPY THE LINES BELOW AND PASTE INTO YOUR CLI TERMINAL *")
print("*            !run in the solution directory!            *")
print("↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓")
print_aws_cli(env_stackname)
print_aws_cli(core_stackname)
print_aws_cli(package_cfn_stackname)
print(f"pipenv run python3 functions/pipeline/clean-up-efs-cli.py {domain_id}")

<div class="alert alert-info"> 
Now you can close Studio and run the generated CLI commands in your terminal.
</div>